<a href="https://colab.research.google.com/github/ucfilho/Codigos_Teste/blob/master/ABCOptim_jun_08_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from random import randint

In [0]:
'''Rosembrock Function'''
def Fun(x):
    fun=0
    a=1.0
    b=100.0
    fun = (a-x[0])**2 + b*(x[1]-x[0]**2)**2
    return fun

In [0]:
''' 
FOBJ gera vetorialmente os valores com a funcao objetivo escalar
em outras palavras gera para populacao o valor da funcao 
a ser otimizada
'''
def FOBJ(X):
    rows = X.shape[0]
    fobj=np.zeros(rows)
    for i in range(rows):
        fobj[i]=Fun(X[i,])
    return fobj

In [0]:
'''Enxame retorna a populacao aleatoria com todos valores entre MIN e MAX'''
def Enxame(PAR,NPAR,MAX,MIN):
    x=np.zeros((NPAR, len(MAX)))
    for j in range(len(MAX)):
        for i in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

In [0]:
''' 
Fitness gera vetorialmente os valores com a funcao fitness
equivale a quantidade de nectar da fonte
'''
def FIT(X):
    rows = X.shape[0]
    fit=np.zeros(rows)
    fit=np.copy(FOBJ(X))
    for i in range(rows):
        if(fit[i]>=0):
            fit[i]=1/(fit[i]+1)
        else:
            fit[i]=1+abs(fit[i])
    return fit

In [0]:
'''
probablilidade usada para roleta q atualizacao das onlookers 
semelhante ao genetico
'''

def PROB(x):
    rows=x.shape[0]
    fit=np.zeros(rows)
    prob=np.zeros(rows)
    fit=FIT(x)
    fit_ref=max(fit)
    
    for i in range(rows):
        prob[i] = 0.9*fit[i]/fit_ref+0.1

    return prob

In [0]:
'''
Obtem a melhor escolha greedy search
'''
def GetBest(x,xbest):
    ycal=FOBJ(x)
    best=np.argmin(ycal)
    yref=Fun(xbest)
    if(yref<ycal[best]):
        GBEST=xbest
    else:
        GBEST=x[best,]
    return GBEST
    

In [0]:
'''
employed bee phase
'''
def EmployedBee(xo,x,MAX,MIN) :
    NPAR = xo.shape[0]
    PAR= xo.shape[1]
    V=np.zeros((NPAR, len(MAX)))

    for j in range(PAR):
        for i in range(NPAR):
            fi=np.random.uniform(low=-1.0, high=1.0, size=None)
            rd=randint(0, (NPAR-1))
            V[i,j]=xo[i,j]+(xo[i,j] -x[rd,j])*fi # candidata a solucao
            if (V[i,j]>MAX[j]):
              V[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
            if (V[i,j]<MIN[j]):
              V[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
              
    YCAL=FOBJ(xo)
    YV=FOBJ(V)
    for i in range(NPAR):
        if(YV[i]<YCAL[i]):
            x[i,]=np.copy(V[i,])

    return x

In [0]:
'''
onlooker bee phase
'''
def OnlookerBee(xo,x): #OnlookerBee(xo,x_Employed)
    rows = xo.shape[0]
    cols = xo.shape[1]
    prob=PROB(x)
    prob_ref=max(prob)
    #best=np.argmax(prob)
    
    for i in range(rows):
      prob_ref=np.random.uniform(low=0, high=1.0, size=None)
      rd=randint(0, (NPAR-1))
      if(prob[i] > prob_ref):  
        fi=np.random.uniform(low=-1.0, high=1.0, size=None)
        Xmi=xo[i,]+fi*(xo[i,]-x[rd,])
        xo[i,]=np.copy(Xmi)
    return xo

In [0]:
'''
scout bee phase
'''
#def ScoutBee(trial,x,ntrail,MAX,MIN):
def ScoutBee(x,MAX,MIN):
    PAR = x.shape[0]
    NPAR = x.shape[1]
    for i in range(PAR):
        for j in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

In [0]:

'''
Metodo que alterna e coordena o uso employers,onlooker, scout bees
e greedy search

'''
def ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN):

    trial=1 # inicializa contador p abandonar fonte de alimento
    xbest=Enxame(PAR,1,MAX,MIN)[0,] #inicializa xbest
    xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee-parte 1
    vbest_old=Fun(xbest)
    vbest_new=Fun(xbest)
    
    for i in range(ITE):
      x=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee-parte 1
      if(vbest_new==vbest_old):
        trial=trial+1
      if(trial==ntrail):
        xo=Enxame(PAR,NPAR,MAX,MIN) # abandona a fonte de alimento antiga
        trial=0
      x=EmployedBee(xo,x,MAX,MIN)
      xbest=GetBest(x,xbest)
      vbest_new=Fun(xbest)
      xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa para OnlookerBee
      x=OnlookerBee(xo,x)
      xbest=GetBest(x,xbest)
      xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa ScoutBee ( a rigor so esta linha basta)
      xbest=GetBest(xo,xbest)
    return xbest, x  

In [12]:
NPAR=50 #PARTICULAS (fontes de comida)
ITE=200 #ITERACOES
trial=np.zeros(NPAR)
PAR=2 #NUM DE PARAMETROS A SER OTIMIZADOS
MAX=[2,2] # MAXIMO DE CADA PARAMETRO
MIN=[-2,-2] # MINIMO DE CADA PARAMETRO
ntrail=25 #numero de buscas ate abandonar uma fonte de alimento

for i in range(6):
    print("resolucao",i+1," ")
    xbest, x =ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN)
    print("vetor",xbest,"funcao", Fun(xbest),"\n")

    

resolucao 1  
vetor [1.0419623  1.08851294] funcao 0.0025603092165273713 

resolucao 2  
vetor [1.02582631 1.05434984] funcao 0.001079180737395432 

resolucao 3  
vetor [0.99547735 0.98983671] funcao 0.00015005716648843864 

resolucao 4  
vetor [1.0158384  1.03129796] funcao 0.0002905052761303945 

resolucao 5  
vetor [1.01613464 1.0281106 ] funcao 0.0022130775183795753 

resolucao 6  
vetor [1.01822066 1.03723084] funcao 0.0003529245462276041 



In [14]:
Num=10
trial=np.zeros(Num)
print(trial)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
